In [ ]:
import torch
import time
import numpy as np
from model import EndoWaveNet  # 假设你把上一段代码保存为了 model.py

def measure_inference_speed(model, height=256, width=256, batch_size=1, device='cuda', iterations=300):
    """
    标准的 FPS 测速函数
    """
    # 1. 准备数据
    dummy_input = torch.randn(batch_size, 3, height, width).to(device)
    
    # 2. 初始化 CUDA 事件 (用于精准计时)
    starter = torch.cuda.Event(enable_timing=True)
    ender = torch.cuda.Event(enable_timing=True)
    
    # 3. 预热 (Warm-up)
    # GPU 在刚开始运行时需要初始化缓存，前几次运行通常很慢且不稳定
    # 我们先空跑 50 次，让 GPU 进入状态
    print(f"🔥 正在预热 GPU (Warm-up)...")
    with torch.no_grad():
        for _ in range(50):
            _ = model(dummy_input)
    
    # 4. 正式测速
    print(f"⏱️ 开始测速 (运行 {iterations} 次, Batch={batch_size}, Size={height}x{width})...")
    timings = np.zeros((iterations, 1))
    
    with torch.no_grad():
        for rep in range(iterations):
            starter.record()
            
            # === 运行模型 ===
            _ = model(dummy_input)
            # =============
            
            ender.record()
            
            # 等待 GPU 完成当前这轮计算
            torch.cuda.synchronize()
            
            # 记录时间 (毫秒)
            curr_time = starter.elapsed_time(ender)
            timings[rep] = curr_time

    # 5. 计算结果
    mean_syn = np.sum(timings) / iterations
    std_syn = np.std(timings)
    fps = (1000.0 / mean_syn) * batch_size # 1000ms / 平均耗时 * Batch数
    
    return mean_syn, std_syn, fps

if __name__ == "__main__":
    # ================= 配置区域 =================
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    H, W = 256, 256  # 输入分辨率
    # ===========================================

    print(f"=======================================")
    print(f"🚀 EndoWaveNet 极速基准测试")
    print(f"   Device: {DEVICE} ({torch.cuda.get_device_name(0) if DEVICE=='cuda' else 'CPU'})")
    print(f"=======================================")

    # 1. 加载模型
    model = EndoWaveNet().to(DEVICE)
    model.eval() # 必须开启 eval 模式 (关闭 Dropout/BatchNorm 更新)

    # 打印参数量
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"📦 模型参数量: {params/1000:.2f} K")

    # 2. 测试不同 Batch Size 的表现
    # 通常 Batch 越大，吞吐量(FPS)越高，但延迟也越高
    batch_sizes = [1, 4, 8, 16]
    
    print(f"\n📊 测速结果:")
    print(f"{'Batch':<8} | {'Resolution':<12} | {'Latency (ms)':<15} | {'FPS':<10}")
    print("-" * 55)

    for b in batch_sizes:
        try:
            mean_time, _, fps = measure_inference_speed(
                model, 
                height=H, 
                width=W, 
                batch_size=b, 
                device=DEVICE
            )
            print(f"{b:<8} | {H}x{W:<8} | {mean_time:.2f} ms        | {fps:.2f}")
        except RuntimeError as e:
            print(f"{b:<8} | {H}x{W:<8} | OOM (显存不足)    | -")
            break

    print("-" * 55)
    print("✅ 测试完成！")